In [1]:
# Sample code from the TorchVision 0.3 Object Detection Finetuning Tutorial
# http://pytorch.org/tutorials/intermediate/torchvision_tutorial.html


In [2]:
import os
import numpy as np
import torch
import torch.utils.data
from PIL import Image

import pandas as pd
# Create dataset based on BCCD 
# Using https://pytorch.org/tutorials/intermediate/torchvision_tutorial.html as reference
class BCCDDataset(torch.utils.data.Dataset):
    def __init__(self, root='/home/sarayu941/blood_imaging/BCCD_Dataset', transforms=None):
        self.root = root
        self.transforms = transforms
        # load all image files, sorting them to
        # ensure that they are aligned
        self.imgs = list(sorted(os.listdir(os.path.join(root, 'BCCD/JPEGImages/'))))
        self.df = pd.read_csv(os.path.join(root, 'test.csv'))

    def __getitem__(self, idx):
        # Load image
        img_path = os.path.join(self.root, 'BCCD/JPEGImages/', self.imgs[idx])
        annotations = self.df.loc[self.df['filename'] == self.imgs[idx]]

        img = Image.open(img_path).convert("RGB")

        boxes = []
        labels = []

        # Convert celltypes to class index
        celltypes = {
            "RBC": 1, 
            "WBC": 2, 
            "Platelets" : 3, 
            "Background": 0}
        
        for index, row in annotations.iterrows():
            boxes.append([row.xmin, row.ymin, row.xmax, row.ymax])
            labels.append(celltypes[row.cell_type])

        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        labels = torch.as_tensor(labels, dtype=torch.int64)
        one_hot = torch.nn.functional.one_hot(labels, num_classes=len(celltypes))

        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        image_id = torch.tensor([idx])
        
        target = {}
        target["boxes"] = boxes
        target["labels"] = one_hot
        target["image_id"] = image_id
        target["area"] = area
        
        if self.transforms is not None:
            img, target = self.transforms(img, target)

        return img, target

    def __len__(self):
        return len(self.imgs)

In [3]:
dataset = BCCDDataset()
dataset[0]

(<PIL.Image.Image image mode=RGB size=640x480>,
 {'boxes': tensor([[260., 177., 491., 376.],
          [ 78., 336., 184., 435.],
          [ 63., 237., 169., 336.],
          [214., 362., 320., 461.],
          [414., 352., 506., 445.],
          [555., 356., 640., 455.],
          [469., 412., 567., 480.],
          [  1., 333.,  87., 437.],
          [  4., 406.,  95., 480.],
          [155.,  74., 247., 174.],
          [ 11.,  84., 104., 162.],
          [534.,  39., 639., 139.],
          [547., 195., 640., 295.],
          [388.,  11., 481., 111.],
          [171., 175., 264., 275.],
          [260.,   1., 374.,  83.],
          [229.,  91., 343., 174.],
          [ 69., 144., 184., 235.],
          [482., 131., 594., 230.],
          [368.,  89., 464., 176.]]),
  'labels': tensor([[0, 0, 1, 0],
          [0, 1, 0, 0],
          [0, 1, 0, 0],
          [0, 1, 0, 0],
          [0, 1, 0, 0],
          [0, 1, 0, 0],
          [0, 1, 0, 0],
          [0, 1, 0, 0],
          [0, 1, 0,

In [4]:
#Fine tuning from a pretrained model
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

# load a model pre-trained pre-trained on COCO
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

# replace the classifier with a new one, that has
# num_classes which is user-defined
num_classes = 4  # 0 Background, 1 RBC, 2 WBC 3 Platelets
# get number of input features for the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features
# replace the pre-trained head with a new one
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes) 

/home/sarayu941/miniconda3/envs/py39/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/home/sarayu941/miniconda3/envs/py39/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [5]:
#An Instance segmentation model for BCCDDataset
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor

      
def get_instance_segmentation_model(num_classes):
    # load an instance segmentation model pre-trained on COCO
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)

    # get the number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    # now get the number of input features for the mask classifier
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    # and replace the mask predictor with a new one
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,
                                                       hidden_layer,
                                                       num_classes)

    return model

In [6]:
# Data augmentation functions
from engine import train_one_epoch, evaluate
import utils
import transforms as T


def get_transform(train):
    transforms = []
    # converts the image, a PIL image, into a PyTorch Tensor
    transforms.append(T.PILToTensor())
    if train:
        # during training, randomly flip the training images
        # and ground-truth for data augmentation
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)

In [7]:
# Test forward function
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
dataset = BCCDDataset('/home/sarayu941/blood_imaging/BCCD_Dataset', get_transform(train=True))
#dataset = BCCDDataset('/home/sarayu941/blood_imaging/BCCD_Dataset')
data_loader = torch.utils.data.DataLoader(
    dataset, batch_size=2, shuffle=True, num_workers=4,
    collate_fn=utils.collate_fn
)


/home/sarayu941/miniconda3/envs/py39/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/home/sarayu941/miniconda3/envs/py39/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/home/sarayu941/miniconda3/envs/py39/lib/python3.9/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that

In [10]:
# For Training
images,targets = next(iter(data_loader))
images = list(image for image in images)
targets = [{k: v for k, v in t.items()} for t in targets]

In [11]:
output = model(images,targets)   # Returns losses and detections
       # Returns predictions

RuntimeError: 0D or 1D target tensor expected, multi-target not supported

In [ ]:
# For inference
model.eval()
x = [torch.rand(3, 300, 400), torch.rand(3, 500, 400)]
predictions = model(x)    